# Get Events and Stations

In this notebook we will get the Groningen events and stations.  We can use a Bounding box to limit the region for which we get events and stations as well.

1. Open Groningen shape-file (provided by KNMI)
1. Plot the shape-file
1. Get all the events
1. Plot an overlay of the events with the shape-file
1. Make a BoundingBox. Start with the bbox for the shape-file
1. Shrink, translate, and rotate the bbox to the desired region
1. Get the events again, but this time only get the events within the bbox
1. Plot the bbox region events
1. Get the stations for each of the bbox-events that are also contained within the bbox
1. Plot all the bbox-events and all stations with markers relating to whether or not the station is in or outside to bbox as well as if there is data for that station-event pair.

### Step 0

Read in Groningen Field shapefile.

In [ ]:
import shapefile as sf

ifqn = './data_in/GroningenFieldShapeFile/Groningen_field'

gron_sf = sf.Reader(ifqn)
print('gron_sf:',gron_sf)

### Step 1

Plot the Groningen Field shape

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

#get coordinates for the Shape-File
s = gron_sf.shape(0)
shape_xy = np.asarray(s.points)

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
title_str = 'Groningen Field'
ax.set_title(title_str)
plt.show()



### Step 2

Get Groningen events

In [ ]:
from gnam.events.gevents import gevents as gevents

# Fetch the events using FDSN
print('Getting Events via FDSN')
min_magnitude = 2.0
events = gevents(min_magnitude)
print()

# Show events
print('Events:\n',events)

### Step 3 

Plot overlay of all the "North" Nederland events on the Groningen Field 

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

#get coordinates for events withing the Bounding Box
event_coords = events.getIncCoords()

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.scatter(event_coords[:,0],event_coords[:,1],s=80,c='red',marker='*',zorder=2)
title_str = '%d Events' %(event_coords.shape[0])
ax.set_title(title_str)
plt.show()



### Step 4

Use BoundingBox and then only get events within the bbox. This features allows one to work with a small model.  This bounding box can then be used to slice a volume from the model, or the bbox from a sliced model, can be used to get related events.

In [ ]:
from gnam.model.bbox import bbox as bb

# construct a bounding box form the "bounding box" field in the shape-file.
print('gron_sf.bbox:',gron_sf.bbox)

c_loop = np.array([[gron_sf.bbox[0],gron_sf.bbox[1]],[gron_sf.bbox[0],gron_sf.bbox[3]],
                   [gron_sf.bbox[2],gron_sf.bbox[3]],[gron_sf.bbox[2],gron_sf.bbox[1]],
                   [gron_sf.bbox[0],gron_sf.bbox[1]]])

gf_bbox = bb(c_loop)

### Step 5

Overlay bbox on the Groningen Field shape and will all the events

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.plot(gf_bbox.getCLoop()[:,0],gf_bbox.getCLoop()[:,1],c='blue',zorder=1)
ax.scatter(event_coords[:,0],event_coords[:,1],s=80,c='red',marker='*',zorder=2)
title_str = '%d Events with Bbox' %(event_coords.shape[0])
ax.set_title(title_str)
plt.show()



### Step 6

Resize, rotate and translate the bbox to cover a region of interest, and then plot it

In [ ]:

# first we shrink and create x and y coordinates for a new bbox
vl = np.array([0,0.87*(gron_sf.bbox[3]-gron_sf.bbox[1])])
#dvl = ((0.8575*(gron_sf.bbox[3]-gron_sf.bbox[1]))**2)**0.5
scl = (0.4)*0.8575
dvl = ((scl*(gron_sf.bbox[3]-gron_sf.bbox[1]))**2)**0.5
nvl = dvl//100 + 1
y = np.arange(nvl)*100
print('nvl:',nvl)

#shrink and create x coordinates for slicing box
vb = np.array([0.85*(gron_sf.bbox[2]-gron_sf.bbox[0]),0])
#dvb = ((0.8625*(gron_sf.bbox[2]-gron_sf.bbox[0]))**2)**0.5
scb = (2/3)*0.8625
dvb = ((scb*(gron_sf.bbox[2]-gron_sf.bbox[0]))**2)**0.5
nvb = dvb//100 + 1
x = np.arange(nvb)*100
print('nvb:',nvb)

#create set of xy coordinates for slicing box
xy = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
print('xy.shape:',xy.shape)



# Now we will create the bbox and then translate it to the origin
#global origins
xy_xmin = np.min(x)
xy_xmax = np.max(x)
xy_ymin = np.min(y)
xy_ymax = np.max(y)

sc_loop = np.array([[xy_xmin,xy_ymin],[xy_xmin,xy_ymax],
                   [xy_xmax,xy_ymax],[xy_xmax,xy_ymin],
                   [xy_xmin,xy_ymin]])

sgf_bbox = bb(sc_loop)
sgf_bbox.translate(gron_sf.bbox[0],gron_sf.bbox[1]) #Translate small bbox to original bbox origin



# Next we will rotate the bbox 
sgf_bbox.rotate(30)


# Next we will translate the bbox to our desired location
xshift = 5800  
yshift = 16800  
sgf_bbox.translate(xshift,yshift)


#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='blue',zorder=1)
ax.scatter(event_coords[:,0],event_coords[:,1],s=80,c='red',marker='*',zorder=2)
title_str = '%d Events with Bbox' %(event_coords.shape[0])
ax.set_title(title_str)
plt.show()



### Step 7

Pickle bounding box

In [ ]:
from gnam.model.bbox import bbox as bb
import pickle

ofqn = './event_and_station_data/bbox_reduced_region.pickle' 

print('Pickling Bbox')
f = open(ofqn, 'wb')
pickle.dump(sgf_bbox, f)
f.close()

print('Output File:\n',ofqn)

### Step 8

We can now used the bounding box to get events that are only within the box, and of course, we will plot them.

In [ ]:
from gnam.events.gevents import gevents as gevents

# Fetch the events using FDSN
print('Getting Events within Bbox via FDSN')
min_magnitude = 2.0
bbox_events = gevents(min_magnitude,bbox=sgf_bbox)
print()

# Show events
print('Bbox Events:\n',bbox_events)
print()

# Get event coordinates
event_coords = bbox_events.getIncCoords()

# Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='blue',zorder=1)
ax.scatter(event_coords[:,0],event_coords[:,1],s=80,c='red',marker='*',zorder=2)
title_str = '%d Events only inside of the Bbox' %(event_coords.shape[0])
ax.set_title(title_str)
plt.show()


### Step 9

Pickle the Groningen events that are within the bbox

In [ ]:
from gnam.events.gevents import gevents as gevents
import pickle

ofqn = './event_and_station_data/bbox_events.pickle' 

print('Pickling Events')
f = open(ofqn, 'wb')
pickle.dump(bbox_events, f)
f.close()
print(ofqn)
    

### Step 10 

Unpickle the above events

In [ ]:
from gnam.events.gevents import gevents as gevents

ifqn = './event_and_station_data/bbox_events.pickle'
print('Opening:',ifqn)
print()

f = open(ifqn, 'rb')
dill_events = pickle.load(f)
f.close()
    
print()
print(events)

### Step 11

Plot the bbox and events again for QC'ing

In [ ]:
from gnam.events.gevents import gevents as gevents

# Get event coordinates
event_coords = dill_events.getIncCoords()

# Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='blue',zorder=1)
ax.scatter(event_coords[:,0],event_coords[:,1],s=80,c='red',marker='*',zorder=2)
title_str = '%d Events only inside of the Bbox' %(event_coords.shape[0])
ax.set_title(title_str)
plt.show()


### Step 12

Next we can get station data for each event withing the bounding box. The "stations" contain the traces for each station and all components.  For more information see the ObsPy documentation

NOTE: This can take some time. (maybe 3-4 hours)

In [ ]:
from gnam.events.gstations import gstations as gstations

print('Getting Station Traces via Events')
# Note: that the events gathered above are used. tend = number of seconds to get
straces = gstations(dill_events,tend=17.0) 
print()

# Show events
print('Stations:\n',straces)
    


### Step 13

Compress and pickle the stations associated with the events in the bounding box

In [ ]:
from gnam.events.gstations import gstations as gstations

ofqn = './event_and_station_data/bbox_stations.pickle'

print('Pickling Station Traces')
f = open(ofqn, 'wb')
pickle.dump(straces, f)
f.close()

print()
print('Output File:\n',ofqn)

### Step 14

Unpickle the stations associated with the events in the bounding box

In [ ]:
from gnam.events.gstations import gstations as gstations

# Copy ofqn from above or use your own
ifqn = './event_and_station_data/bbox_stations.pickle'
print('Input File:\n',ifqn)
print()

print('Unpickling Station Traces')
f = open(ifqn, 'rb')
dill_straces = pickle.load(f)
f.close()

print()
print('Stations:\n',dill_straces)

### Step 15

Plot event and stations within the bbox overlaid on Groningen Field

In [ ]:
# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
from gnam.events.munge.knmi import correct_station_depths as csd_f
dill_straces.correct_stations(csd_f)


#get event and borhole keys used for indexing
ekeys = dill_straces.getEventKeys()
bkeys = dill_straces.getBoreholeKeys()

#Plot seuence of events with stations 
for ie in ekeys:
    # coordinates for stations that are in the bounding box
    xy3 = dill_straces.getIncStationCoords(ie,bkeys[0]) #station code G##3
    xy4 = dill_straces.getIncStationCoords(ie,bkeys[1]) #station code G##4
    
    # coordinates for stations that are G-stations but outside the bounding box
    ex_xy3 = dill_straces.getExcStationCoords(ie,bkeys[0]) #station code G##3
    ex_xy4 = dill_straces.getExcStationCoords(ie,bkeys[1]) #station code G##4
    
    # coordinates for stations that are inside the bounding box but there is no data
    er_xy3 = dill_straces.getErrStationCoords(ie,bkeys[0]) #station code G##3
    er_xy4 = dill_straces.getErrStationCoords(ie,bkeys[1]) #station code G##4

    fig, ax = plt.subplots(1,figsize=(8,8))
    
    #Groningen Field Shape-File
    ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
    
    #Bounding Box
    ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='green',zorder=1)
    
    #Events (reuse event coordinates from cell further above)
    ax.scatter(event_coords[ie,0],event_coords[ie,1],s=90,c='red',marker='*',zorder=5)
    
    #Included stations
    ax.scatter(xy3[:,0],xy3[:,1],s=50,c='blue',marker='v',zorder=3)
    ax.scatter(xy4[:,0],xy4[:,1],s=100,c='gray',marker='o',zorder=2)
    
    #Excluded stations
    ax.scatter(ex_xy3[:,0],ex_xy3[:,1],s=80,c='lightgray',marker='1',zorder=4)
    ax.scatter(ex_xy4[:,0],ex_xy4[:,1],s=100,c='lightgray',marker='2',zorder=3)
    
    #Stations without data
    ax.scatter(er_xy3[:,0],er_xy3[:,1],s=50,c='yellow',marker='v',zorder=4)
    ax.scatter(er_xy4[:,0],er_xy4[:,1],s=100,c='gray',marker='o',zorder=3)
    
    origin_time = dill_events[ie].origins[0].time
    mag = events[ie].magnitudes[0].mag
    title_str = 'Event-%d, Origin Time: %s, Magnitude: %1.2f' %(ie,str(origin_time),mag)
    ax.set_title(title_str)
    plt.show()